# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [38]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [39]:
# checking our current working directory
print(os.getcwd())

# our current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

#  a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
#  file path and roots joined with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(filepath,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [40]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [41]:
# check the number of rows in our csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Completing the Apache Cassandra coding portion of the project. 

## Now we're ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [42]:
# This makes a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries - a session
session = cluster.connect()

#### Create Keyspace

In [43]:
session.execute("""
create keyspace if not exists events
with replication =
{'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")


#### Set Keyspace

In [44]:

session.set_keyspace('events')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra we model the database tables on the queries you want to run.

## Queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [45]:
#for this query the primary key is the sessionid as by sessionid we are linking this table, iteminsession is the clustering column

#we create a tanle here and are initialising with column names

query = "create table if not exists song_by_sessionid_and_iteminsession"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, primary key((sessionId), itemInsession) )"
session.execute(query)

                    

In [46]:
#values are inserted here, some sample values in Apache Cassandra format or CQL

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_by_sessionid_and_iteminsession (sessionId, itemInSession, artist, song, length)"
        query = query + "values (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [47]:
#after above operations of creating table and inserting data we hereby test it for validaion

query = "select artist, song, length from song_by_sessionid_and_iteminsession where sessionId=338 and itemInsession=4"

rows=session.execute(query)

for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [ ]:
QUERY 2

In [48]:
#CREATING THE TABLE
query = "create table if not exists song_by_userid_and_sessionid"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, user text, primary key((userId, sessionId), itemInsession))"
session.execute(query)

In [49]:

#INSERT AND ROW ADDITION OPERATIONS
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        user=line[1]+" "+ line[4]
        query = "insert into song_by_userid_and_sessionid (userId, sessionId, itemInSession, artist, song, user)"
        query = query + "values (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], user, line[9]))
        

In [50]:

#DATA VALIDATION THOUGH SELECT OPERATION AND WHERE CLAUSE

query = "select artist, song, user from song_by_userid_and_sessionid where sessionId=182 and userId=10"

rows=session.execute(query)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song='Sylvie Cruz', user="Keep On Keepin' On")
Row(artist='Three Drives', song='Sylvie Cruz', user='Greece 2000')
Row(artist='Sebastien Tellier', song='Sylvie Cruz', user='Kilometer')
Row(artist='Lonnie Gordon', song='Sylvie Cruz', user='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)')


In [ ]:
QUERY 3

In [51]:

#CREATING THE TABLE
query = "create table if not exists user_by_song"
query = query + "(song text, userId int, user text, primary key((song), userId))"
session.execute(query)

In [52]:

#INSERTING AND ADDITION OF ROWS
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        user=line[1]+" "+line[4]
        query = "insert into user_by_song (song, userId, user)"
        query = query + "values (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), user))
        

In [53]:
        
#DATA VALIDATION THROUGH SELECT STATEMENT AND WHERE CLAUSE
query = "select song, user from user_by_song where song='All Hands Against His Own' "

rows=session.execute(query)

for row in rows:
    print(row)

Row(song='All Hands Against His Own', user='Jacqueline Lynch')
Row(song='All Hands Against His Own', user='Tegan Levine')
Row(song='All Hands Against His Own', user='Sara Johnson')


### Drop the tables before closing out the sessions

In [54]:
#DROPPING TABLES HERE

query="drop table song_by_sessionid_and_iteminsession"
session.execute(query)
query="drop table song_by_userid_and_sessionid"
session.execute(query)
query="drop table user_by_song"
session.execute(query)

### Close the session and cluster connection¶

In [56]:
session.shutdown()
cluster.shutdown()